# Parking in Berkeley
Street parking in many major cities is a fraught process. We would like to make it a little bit easier.
* Paths of patrol
* General ticketing stats

In [85]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pdfplumber
import pandas as pd
from tqdm import tqdm  # For progress tracking

def extract_pdf_tables_optimized(pdf_path, chunk_size=10):
    all_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)
        print(f"Total pages: {total_pages}")
        
        # Get headers from first page
        first_page = pdf.pages[0]
        first_table = first_page.extract_table()
        headers = first_table[0] if first_table else None
        
        # Process pages in chunks to manage memory
        for i in tqdm(range(0, total_pages, chunk_size)):
            chunk_pages = pdf.pages[i:min(i + chunk_size, total_pages)]
            
            for page in chunk_pages:
                try:
                    table = page.extract_table()
                    if table:
                        # Skip header row except for first table
                        rows_to_add = table[1:] if i > 0 or page != first_page else table[1:]
                        all_data.extend(rows_to_add)
                except Exception as e:
                    print(f"Error on page {i}: {str(e)}")
                    continue
                
            # Create interim DataFrame to free up memory
            if len(all_data) > 100000:  # Save to disk every 100k rows
                interim_df = pd.DataFrame(all_data, columns=headers)
                interim_df.to_csv(f'parking_data18_part_{i}.csv', index=False)
                all_data = []  # Clear memory
    
    # Final DataFrame creation
    if all_data:  # If there's remaining data
        final_df = pd.DataFrame(all_data, columns=headers)
        final_df.to_csv('parking_data18_final.csv', index=False)
    
    # Combine all parts if we created interim files
    import glob
    csv_files = glob.glob('parking_data18_part_*.csv')
    if csv_files:
        dfs = []
        for file in csv_files:
            dfs.append(pd.read_csv(file))
        if all_data:  # Add the final batch if it exists
            dfs.append(final_df)
        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df.to_csv('parking_data18.csv', index=False)
        
        # Cleanup interim files
        for file in csv_files:
            import os
            os.remove(file)
        
        return combined_df
    else:
        return final_df if 'final_df' in locals() else pd.DataFrame()

# Use the function with progress bar
#df = extract_pdf_tables_optimized('data/PRR_Berkeley_Parking_Ticket_Data/Updated PRR_12.14.2018_2018.pdf')
#print(f"Final DataFrame shape: {df.shape}")

# this code took about 3hrs to run, and finally we have a csv instead. So we are going to use that.
df = pd.read_csv('data/parking_data18_final.csv')

/var/folders/k9/jh1slgvn5m7cfjqr6rcfd7th0000gn/T/ipykernel_41445/4152518084.py:72: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/parking_data18_final.csv')


In [86]:
# Load one full year of parking ticket data (2018) in dataframe
df.columns = df.columns.str.replace('\n', '_')
df.columns = df.columns.str.replace(' ', '_')
df.shape, df.columns

((39882, 17),
 Index(['Issue_Year', 'Issue_Mnth_NBR', 'Ticket_Number', 'Tick_Issue_Date',
        'Tick_Issue_Time', 'Agency', 'Badge_#', 'Tick_Vehicle_Type',
        'Tick_Veh_Model', 'Rp_Plate_Expire_Date', 'Violation_Code', 'Violation',
        'Tick_Meter', 'Tick_Street_Name', 'Tick_Total_Paid', 'Total_Amt_Paid',
        'Rp_Total_Amt_Due'],
       dtype='object'))

In [87]:
# we want to first look for patterns in time and street name
# lets start by converting the time to some sort of numerical value we can work with easier
# first idea: convert time str to int representing minutes out of the day 
# given there are 1440 minutes in the day
# (12:00AM -> 0, 11:59PM -> 1439)
# Option 1: Using direct time string split
mikes_str_expanded = df['Tick_Issue_Time'].str.split(':', expand=True)
df['TIT_MIKES'] = mikes_str_expanded[0].map(lambda x: int(x)) * 60 + mikes_str_expanded[1].map(lambda x: int(x))
df["TIT_MIKES"]

0         651
1         652
2         653
3         653
4         654
         ... 
39877    1062
39878    1063
39879    1067
39880    1068
39881    1072
Name: TIT_MIKES, Length: 39882, dtype: int64

In [88]:
df

,Issue_Year,Issue_Mnth_NBR,Ticket_Number,Tick_Issue_Date,Tick_Issue_Time,Agency,Badge_#,Tick_Vehicle_Type,Tick_Veh_Model,Rp_Plate_Expire_Date,Violation_Code,Violation,Tick_Meter,Tick_Street_Name,Tick_Total_Paid,Total_Amt_Paid,Rp_Total_Amt_Due,TIT_MIKES
0,2018,9,1807789876,09/07/2018,10:51,1.0,841,NaN,NaN,NaN,14.36.030D,STREET SWEEPING,NaN,NINTH,$49.00,$49.00,$0,651
1,2018,9,1807789880,09/07/2018,10:52,1.0,841,NaN,NaN,NaN,14.36.030D,STREET SWEEPING,NaN,NINTH,$49.00,$49.00,$0,652
2,2018,9,1807760454,09/07/2018,10:53,1.0,809,NaN,NaN,NaN,14.40.090(1),PREF AREA 2 HOUR NO,NaN,ACROSS FROM 2342 GRANT,$49.00,$49.00,$0,653
3,2018,9,1807834212,09/07/2018,10:53,1.0,826,NaN,NaN,NaN,14.52.063,NO PAY STATION DMT D,NaN,BOWDITCH,$0.00,$0.00,"$1,145",653
4,2018,9,1807281562,09/07/2018,10:54,1.0,818,NaN,NaN,NaN,14.52.050A,METER TIME EXPIRED,CH2047,CHANNING WAY,$43.00,$43.00,$0,654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39877,2018,12,1808238591,12/13/2018,17:42,1.0,818,NaN,NaN,NaN,14.52.050A,METER TIME EXPIRED,W1503,WALNUT,$0.00,$0.00,$200,1062
39878,2018,12,1808238602,12/13/2018,17:43,1.0,818,NaN,NaN,NaN,5204A(2),EXPIRED TABS READ BA,W1503,WALNUT,$0.00,$0.00,$200,1063
39879,2018,12,1808243841,12/13/2018,17:47,1.0,826,NaN,NaN,NaN,22500E,BLOCKING EXTEND INTO,NaN,MCKINLEY,$0.00,$0.00,$60,1067
39880,2018,12,1808238613,12/13/2018,17:48,1.0,818,NaN,NaN,NaN,14.52.050A,METER TIME EXPIRED,W1453,WALNUT,$0.00,$0.00,$43,1068


In [89]:
df_test = df[(df["Badge_#"] == '808') & (df["Tick_Issue_Date"] == '09/07/2018')]
df_test = df_test[['Tick_Street_Name', 'TIT_MIKES']]

In [90]:
test_streets = df_test['Tick_Street_Name'].unique()
test_streets

array(['BONITA', 'ACROSS FROM 1945 BERKELEY WAY', 'NEXT TO 1905 HENRY',
       'HENRY', 'NEXT TO 1902 HENRY', 'HEARST', 'NEXT TO 2005 HEARST',
       'DELAWARE', 'VIRGINIA', 'ACROSS FROM 1917 BERKELEY WAY',
       'BERKELEY WAY'], dtype=object)

In [91]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderInsufficientPrivileges
import pandas as pd
import time
import re

# Initialize Nominatim with a proper User-Agent
geolocator = Nominatim(user_agent="my_geocoding_project")

# Sample street names (replace this with your DataFrame's data)
test_streets = [
    'BONITA', 
    'ACROSS FROM 1945 BERKELEY WAY', 
    'NEXT TO 1905 HENRY',
    'HENRY', 
    'NEXT TO 1902 HENRY', 
    'HEARST', 
    'NEXT TO 2005 HEARST',
    'DELAWARE', 
    'VIRGINIA', 
    'ACROSS FROM 1917 BERKELEY WAY',
    'BERKELEY WAY'
]

# Common street suffixes
street_suffixes = ["St", "Ave", "Blvd", "Rd", "Dr", "Way", "Ct", "Pl", "Ln"]

# Function to clean up address descriptions
def clean_address(address):
    address = address.strip().upper()
    if "NEXT TO" in address or "ACROSS FROM" in address:
        match = re.search(r"(\d+)\s(.+)", address)
        if match:
            number, street = match.groups()
            return f"{number} {street}, Berkeley, CA"
    return f"{address}, Berkeley, CA"

# Function to validate longitude
def validate_longitude(lat, lon):
    if lon is None or lon > -121 or lon < -123:  # Berkeley's longitude range
        return False
    return True

# Function to generate address variations with suffixes
def generate_variations(base_address):
    return [f"{base_address} {suffix}, Berkeley, CA" for suffix in street_suffixes]

# Function to retry geocoding with suffixes
def retry_geocode_with_suffixes(base_address):
    variations = generate_variations(base_address)
    for variation in variations:
        lat, lon = geocode_address(variation)
        if validate_longitude(lat, lon):  # Only accept valid results
            return lat, lon, variation
    return None, None, None

# Function to geocode an address
def geocode_address(address):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except (GeocoderTimedOut, GeocoderInsufficientPrivileges):
        return None, None

# Main workflow
results = []
for street in test_streets:
    cleaned_address = clean_address(street)
    lat, lon = geocode_address(cleaned_address)
    if not validate_longitude(lat, lon):  # Retry for invalid results
        base_address = cleaned_address.split(",")[0]  # Remove ", Berkeley, CA"
        lat, lon, corrected_address = retry_geocode_with_suffixes(base_address)
        results.append({
            "Original Street": street,
            "Cleaned Address": corrected_address or cleaned_address,
            "Latitude": lat,
            "Longitude": lon
        })
    else:
        results.append({
            "Original Street": street,
            "Cleaned Address": cleaned_address,
            "Latitude": lat,
            "Longitude": lon
        })
    time.sleep(1)  # Rate-limiting

# Convert results to DataFrame
df_results = pd.DataFrame(results)

In [80]:
df_results

,Original Street,Cleaned Address,Latitude,Longitude
0,BONITA,"BONITA, Berkeley, CA",37.873136,-122.272059
1,ACROSS FROM 1945 BERKELEY WAY,"1945 BERKELEY WAY, Berkeley, CA",37.872896,-122.271275
2,NEXT TO 1905 HENRY,"1905 HENRY St, Berkeley, CA",37.873548,-122.269572
3,HENRY,"HENRY St, Berkeley, CA",37.884387,-122.271049
4,NEXT TO 1902 HENRY,"1902 HENRY St, Berkeley, CA",37.873381,-122.269854
5,HEARST,"HEARST, Berkeley, CA",37.873675,-122.254241
6,NEXT TO 2005 HEARST,"2005 HEARST, Berkeley, CA",37.873842,-122.270645
7,DELAWARE,"DELAWARE, Berkeley, CA",37.871340,-122.294724
8,VIRGINIA,"VIRGINIA St, Berkeley, CA",37.871315,-122.304137
9,ACROSS FROM 1917 BERKELEY WAY,"1917 BERKELEY WAY, Berkeley, CA",37.872677,-122.272348


In [81]:
import folium

# Convert dataframe columns to a list of (lat, lon) tuples
coordinates = list(zip(df_results["Latitude"], df_results["Longitude"]))

# Create a map centered at the first coordinate
my_map = folium.Map(location=coordinates[0], zoom_start=6)

# Add markers and a polyline
for lat, lon in coordinates:
    folium.Marker(location=[lat, lon]).add_to(my_map)
folium.PolyLine(locations=coordinates, color="blue").add_to(my_map)

my_map  # Display inline in a Jupyter notebook